In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from models import AlexNet, LeNet
from keras.applications.vgg16 import VGG16

from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import backend as K
K.set_image_data_format('channels_last')

from keras.callbacks import ReduceLROnPlateau
from keras.layers import BatchNormalization, Activation, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.models import Model

import glob

import tensorflow as tf

Using TensorFlow backend.


In [2]:
activations = ['relu', 'lrelu']
img_treats=['image-treat-1', 'image-treat-2', 'image-treat-3', 'abordagem-4', 'abordagem-5', 'abordagem6', 'abordagem7', 'abordagem9', 's']
nets = ['lenet', 'alexnet', 'vgg16']

activation, img_treat, net = activations[1], img_treats[7], nets[2]
activation, img_treat, net

('lrelu', 'abordagem9', 'vgg16')

In [3]:
test_dataset = BatchGenerator(box_output_format=['class_id'])
test_dataset.parse_csv(labels_filename='dataset/csv/imdb_csv/imdb_age_regression_test_split_47950-70-10-20.csv', 
                        images_dir='dataset/imdb-hand-crop/',
                        input_format=['image_name', 'class_id'])

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 90
melhor_epoca='08'
train_batch_size = 128
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:

test_generator = test_dataset.generate(batch_size=train_batch_size,
                                       shuffle=shuffle,
                                       ssd_train=ssd_train,
                                       divide_by_stddev = 225,
                                       #equalize=True,
                                       returns={'processed_labels'},
                                       resize=(img_height, img_width))

In [5]:
print("Number of images in the dataset:", test_dataset.get_n_samples())

Number of images in the dataset: 9590


In [6]:
steps = test_dataset.get_n_samples()/train_batch_size

In [7]:
if net == 'alexnet':
    model = AlexNet
elif net =='lenet':
    model = LeNet
elif net == 'vgg16':
    model = VGG16 
csvlogger_name = 'callbacks/'+net +'/age/consolidados/history-regression-' + img_treat + '-' + activation + '.csv'
checkpoint_filename = 'callbacks/'+net+'/age/consolidados/class-weights-' + img_treat + '-' + activation + '.{epoch:02d}-{val_loss:.2f}.hdf5'
csvlogger_name, checkpoint_filename

weights_path = glob.glob('callbacks/' + net + '/age/consolidados/class-weights-' + img_treat + '-' + activation + '*')[0]
#weights_path = glob.glob('callbacks/' + net + '/age/class-weights-' + img_treat + '-' + activation + '.' + melhor_epoca + '*')[0]

base_model = VGG16(include_top=True, weights=None, input_tensor=None, 
                  input_shape=(img_height, img_width, img_depth), 
                  pooling='avg')

base_model.summary()

#alexnet.model.load_weights('callbacks/alexnet/age/weights.24-1658.03.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
base_model.layers.pop()

last = base_model.layers[-1].output

preds = Dense(1)(last)
preds = Activation('relu')(preds)

model = Model(base_model.input, preds)

In [13]:
model.load_weights(weights_path)

In [14]:
optimizer = Adam(lr=0.0000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)

In [15]:
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [21]:
model.evaluate_generator(test_generator, steps=steps, use_multiprocessing=True)

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, block1_conv1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_1/mul/_241}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_203_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated

In [16]:
stats = model.evaluate_generator(test_generator,steps=stepsuse_multiprocessing=True)


sqrt(stats[0]), stats

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, block1_conv1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_1/mul/_241}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_203_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated

In [ ]:
start = weights_path.find(activation)+len(activation)+1
end = start + 2
epoca = int(weights_path[start:end]) + 1

In [ ]:
try:
    df_results = pd.read_csv('graficos/results_teste.csv', index_col=0)

except FileNotFoundError:
    df_results = pd.DataFrame({'Rede':[], 'Ativação':[], 'RMSE':[], 'MAE':[], 'FASE': [], 'Epoca': []})
#df_results = df_results.append({'Rede':net, 'Ativação':activation, 'RMSE': sqrt(stats[0]), 'MAE':stats[1], 'FASE': img_treat, 'Epoca': epoca}, ignore_index=True)

# In[13]:

#df_results.to_csv('graficos/results_teste_' + img_treat + '.csv', index_col=0)
df_results.to_csv('graficos/results_teste.csv')



In [ ]:
df_results

In [ ]:
prev = next(test_generator)


#prev[1]
predict = model.predict(prev[0])


In [ ]:
print(predict[:5])